In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
#  Fornberg formula
#  Input Parameters
#    z            -  location where approximations are to be
#                    accurate
#    x(0:nd)      -  grid point locations, found in x(0:n)
#    nd           -  dimension of x- and c-arrays in calling
#                    program x(0:nd) and c(0:nd, 0:m), respectively
#    m            -  highest derivative for which weights are
#                    sought
#
#  Output Parameter
#    c(0:nd,0:m)  -  weights at grid locations x(0:n) for
#                    derivatives of order 0:m, found in c(0:nd, 0:m)
#
#  References:
#      Generation of Finite Difference Formulas on Arbitrarily
#          Spaced Grids, Bengt Fornberg,
#          Mathematics of compuation, 51, 184, 1988, 699--706,
#          doi: 10.1090/S0025-5718-1988-0935077-0

def weights(z, x, nd, m):
    c1 = 1
    c4 = x[0] - z
    c = np.zeros((nd+1, m+1))
    c[0, 0] = 1
    for i in range(1, nd+1):
        mn = min(i, m)
        c2 = 1
        c5 = c4
        c4 = x[i] - z
        for j in range(0, i):
            c3 = x[i] - x[j]
            c2 = c2*c3
            if j == i-1:
                for k in range(mn, 0, -1):
                    c[i, k] = c1*(k*c[i-1, k-1] - c5*c[i-1, k])/c2
                c[i, 0] = -c1*c5*c[i-1, 0]/c2
            for k in range(mn, 0, -1):
                c[j, k] = (c4*c[j, k] - k*c[j, k-1])/c3
            c[j, 0] = c4*c[j, 0]/c3
        c1 = c2
    return c

In [3]:
def get_yprimes(nleft, m, x, y):
    """Return derivatives of ``y`` up to order ``m`` on grid ``x``.

    Parameters
    ----------
    nleft : int
        Number of stencil points to the left (right uses the same count).
    m : int
        Maximum derivative order. Valid range: ``0 <= m <= 2*nleft``.
    x : np.ndarray
        Grid points.
    y : np.ndarray
        Function values on ``x``.

    Returns
    -------
    derivs : np.ndarray
        Array with shape ``(m+1, npt)`` where ``derivs[p, j]`` is the
        ``p``-th derivative at ``x[j]``.
    """
    npt = len(x)
    nright = nleft
    nd = nleft+nright
    if m < 0 or m > nd:
        raise ValueError(f"m must satisfy 0 <= m <= {nd}, got {m}")

    derivs = np.zeros((m+1, npt))
    xsten = np.zeros(nd+1)
    for j in range(npt):
        z = x[j]
        if j-nleft < 0:
            j0 = 0
            j1 = nd+1
            xsten[0:nd+1] = x[j0:j1]
        elif j-nleft+nd+1 > npt-1:
            j1 = npt
            j0 = j1-nd-1
            xsten[0:nd+1] = x[j0:j1]
        else:
            j0 = j-nleft
            j1 = j0+nd+1
            xsten[0:nd+1] = x[j0:j1]

        c = weights(z, xsten, nd, m)
        for p in range(m+1):
            tmp = 0.
            for k in range(nd+1):
                tmp = tmp+c[k, p]*y[j0+k]
            derivs[p, j] = tmp
    return derivs


In [4]:
def get_newy(nleft, x, y, xnew):
    m = 0
    npt = len(x)
    npt1 = len(xnew)
    ynew = np.zeros(npt1)
    nright = nleft
    nd = nleft+nright
    xsten = np.zeros(nd+1)
    for jjtt in range(npt1):
        tmr = 1e99
        k0 = 0
        for j in range(npt):
            if np.abs(xnew[jjtt]-x[j]) < tmr:
                tmr = np.abs(xnew[jjtt]-x[j])
                k0 = j
        for j in range(k0, k0+1):
            z = xnew[jjtt]
            tmp = 0.
            if j-nleft < 0:
                j0 = 0
                j1 = nd+1
                xsten[0:nd+1] = x[j0:j1]
            elif j-nleft+nd+1 > npt-1:
                j1 = npt
                j0 = j1-nd-1
                xsten[0:nd+1] = x[j0:j1]
            else:
                j0 = j-nleft
                j1 = j0+nd+1
                xsten[0:nd+1] = x[j0:j1]
            c = weights(z, xsten, nd, m)
            for k in range(nd+1):
                tmp = tmp+c[k, 0]*y[j0+k]
            ynew[jjtt] = tmp
    return ynew

In [5]:
def gram_schmidt(V, eps=1e-12):
    """
    Gram-Schmidt orthonormalization.

    Policy for linearly dependent (or near-zero) inputs:
    raise ValueError when the residual norm before normalization is < eps,
    so callers can handle retry/reinitialization explicitly.
    """
    orthogonal = []
    for i in range(len(V)):
        v = copy.deepcopy(V[i])
        for j in range(i):
            v = v - np.dot(orthogonal[j], v) * orthogonal[j]
        norm_v = np.linalg.norm(v)
        if norm_v < eps:
            raise ValueError(
                f"gram_schmidt failed at index {i}: residual norm {norm_v:.3e} < eps={eps:.1e}"
            )
        v = v/norm_v
        orthogonal.append(v)
    return np.array(orthogonal)


In [6]:
def preconditioning(vector):
    npt = len(vector)
    wector = np.zeros(npt)
    for jjtt in range(2):
        wector[:] = vector[:]
        for j in range(1, npt-1):
            vector[j] = 0.9*wector[j]+0.1*(wector[j-1]+wector[j+1])/2.
    return vector

In [7]:
def solver0(nbandi, nleft, x, vpot, eig, vec):
    npt = len(x)
    vec1 = np.zeros((nbandi, npt))
    eig1 = np.zeros(nbandi)
    m = 2
    if False:
        for i in range(nbandi):
            for j in range(nbandi):
                tmp = np.dot(vec[i, :], vec[j, :])
                print(tmp)
    for iitt in range(1000*nbandi):
        for i in range(nbandi):
            for kktt in range(10):
                derivs = get_yprimes(nleft, m, x, vec[i, :])
                dpsi = derivs[1, :]
                hpsi = derivs[2, :]
                hpsi[:] = -0.5*hpsi[:] + vpot[:]*vec[i, :]
                eig[i] = np.dot(vec[i, :], hpsi[:])
                dpsi[:] = hpsi[:]-eig[i]*vec[i, :]
                if np.max(np.abs(dpsi[:]))/np.max(np.abs(vec[i, :])) < 1e-4:
                    break
#           dpsi = preconditioning(dpsi)
                for j in range(i+1):
                    dpsi[:] = dpsi[:]-np.dot(vec[j, :], dpsi[:])*vec[j, :]
                zderivs = get_yprimes(nleft, m, x, dpsi[:])
                z2 = zderivs[2, :]
                z2[:] = -0.5*z2[:] + vpot[:]*dpsi[:]
                tmr = np.dot(dpsi[:], hpsi[:])
                tmq = np.dot(dpsi[:], z2[:])
                theta = 0.5*np.arctan((2*tmr)/(eig[i]-tmq))
                c = np.cos(theta)
                s = np.sin(theta)
                vec[i, :] = c*vec[i, :]+s*dpsi[:]
        if False:
            ind = eig.argsort()
            vec1[:, :] = vec[:, :]
            eig1[:] = eig[:]
            for i in range(nbandi):
                j = ind[i]
                eig[i] = eig1[j]
                vec[i, :] = vec1[j, :]
        if False:
            hh = np.zeros((nbandi, nbandi))
            for j in range(nbandi):
                for i in range(nbandi):
                    derivs = get_yprimes(nleft, m, x, vec[i, :])
                    hpsi = derivs[2, :]
                    hpsi[:] = -0.5*hpsi[:] + vpot[:]*vec[i, :]
                    hh[j, i] = np.dot(vec[j, :], hpsi[:])
            eigenvalues, eigenvectors = np.linalg.eig(hh)
            eig[:] = eigenvalues[:]
            vec1[:, :] = vec[:, :]
            for i in range(nbandi):
                vec[i, :] = 0.
                for k in range(nbandi):
                    vec[i, :] = vec[i, :] + eigenvectors[i, k]*vec1[k, :]
        if True:
            ind = eig.argsort()
            vec1[:, :] = vec[:, :]
            eig1[:] = eig[:]
            for i in range(nbandi):
                j = ind[i]
                eig[i] = eig1[j]
                vec[i, :] = vec1[j, :]
        if True:
            try:
                vec = gram_schmidt(vec)
            except ValueError as exc:
                raise RuntimeError(
                    f"solver0 failed during orthonormalization at iteration {iitt}"
                ) from exc
            if vec.shape != (nbandi, npt):
                raise RuntimeError(
                    f"solver0 received invalid orthonormalized shape {vec.shape}, expected {(nbandi, npt)}"
                )
        if True:
            if iitt % 100 == 0 and iitt > 1:
                print(eig)
    return eig, vec

In [8]:
nbandi = 6
nleft = 2
npt = 100
x = np.zeros(npt)
vpot = np.zeros(npt)
xf = 5.
xs = -5.
for j in range(npt):
    x[j] = xs+(xf-xs)*float(j)/(npt-1)
    vpot[j] = 0.5*x[j]**2
eig = np.zeros(nbandi)
vec = np.zeros((nbandi, npt))
for i in range(nbandi):
    for j in range(npt):
        vec[i, j] = np.random.random()-0.5
    vec[i, :] = vec[i, :]/np.linalg.norm(vec[i, :])
for j in range(npt):
    vec[0, j] = np.exp(-((x[j])/(2.))**2)
vec[0, :] = vec[0, :]/np.linalg.norm(vec[0, :])
vec = gram_schmidt(vec)
eig, vecold = solver0(nbandi, nleft, x, vpot, eig, vec)

[0.49999975 1.49999986 2.49998077 3.55887601 4.4991006  5.47947671]
[0.45742324 1.49999803 2.47655976 3.49994657 4.51579978 5.49982184]
[0.499999   1.49963624 2.4994978  3.49863346 4.49507721 5.47504968]
[0.49999901 1.49999283 2.49997371 3.50026325 4.29730176 5.49788687]
[0.49999902 1.49999315 2.49997477 3.49994432 4.49903339 5.43201253]
[0.49999904 1.4999091  2.49997855 3.49996151 4.49927576 5.49971247]
[0.49999898 1.49999318 2.49909873 3.50037438 4.49985784 5.49971195]
[0.49999917 1.49909689 2.49806225 3.49478092 4.48892529 5.49972821]
[0.49999892 1.4992587  3.49861556 3.49993085 4.49984794 5.49737724]
[0.49999892 1.49999292 2.48779661 3.4999489  4.49989278 5.49857891]
[0.49999892 1.49999347 2.49763406 3.49993246 4.49987297 5.49978109]
[0.49999892 1.49999274 2.49903199 3.47576984 4.49984826 5.49968169]
[0.50000095 1.49999312 2.49997496 3.49993235 4.49988937 5.49971903]
[0.49999893 1.49999267 2.50006638 3.49993069 4.4990241  5.49682065]
[0.49999902 1.49999268 2.49997307 3.49993065 4.4

In [9]:
npt = 200
xnew = np.zeros(npt)
vpot = np.zeros(npt)

eig = np.zeros(nbandi)
vec = np.zeros((nbandi, npt))
for j in range(npt):
    xnew[j] = xs+(xf-xs)*float(j)/(npt-1)
    vpot[j] = 0.5*xnew[j]**2
for i in range(nbandi):
    for j in range(len(xnew)):
        vec[i, j] = np.interp(xnew[j], x, vecold[i, :])
if True:
    for i in range(nbandi):
        for j in range(len(xnew)):
            vec[i, j] = np.interp(xnew[j], x, vecold[i, :])
if False:
    for i in range(nbandi):
        vec[i, :] = get_newy(nleft, x, vecold[i, :], xnew)
vec = gram_schmidt(vec)
eig, vec = solver0(nbandi, nleft, xnew, vpot, eig, vec)

[0.49959458 1.50000172 2.49994466 3.49775366 4.49982041 5.4999308 ]
[0.50000003 1.50000091 2.50001425 3.49999832 4.50264492 5.49995564]
[0.49999996 1.5000002  2.49985292 3.49896892 4.49983659 5.49984672]
[0.49999995 1.4999998  2.50527641 3.4994981  4.49991863 5.50208555]
[0.49999995 1.49995117 2.49993043 3.49999515 4.49999806 5.50006704]
[0.49999995 1.49910235 2.49996769 3.49992277 4.49998657 5.49984711]
[0.49999995 1.50000014 2.52144839 3.49999676 4.49998687 5.49995695]
[0.5        1.50000277 2.49845067 3.49999549 4.49998689 5.49995672]
[0.49999997 1.50000441 2.49686381 3.49999319 4.49998666 5.49995672]
[0.49999998 1.50000136 2.46970364 3.49999506 4.49998663 5.49995668]
[0.49999996 1.49998016 2.50000892 3.49999506 4.49998689 5.49995654]
[0.49999999 1.4996892  2.50000789 3.49999506 4.49998691 5.49995647]
[0.50000015 1.50000026 2.50000222 3.49999505 4.49998694 5.49995718]
[0.50000046 1.49999053 2.49966035 3.49999504 4.49989002 5.49995598]
[0.49997847 1.5000058  2.49999828 3.49999541 4.4